In [2]:
import pandas as pd
import numpy as np
import requests
import json as js

# 0.0 Pulling API Data
## 0.1 Get Unique locations to do Yelp business ID lookup
First we need to get each unique location to try to run them against the yelp API to see if we can get a business id

In [5]:
healthdf = pd.read_csv('/home/michael/Downloads/Food_Inspections.csv')

In [6]:
healthdf.shape

(164683, 17)

In [7]:
healthdf.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2144504,TAQUERIA EL RANCHITO,TAQUERIA EL RANCHITO,60184.0,Restaurant,Risk 1 (High),2829 N MILWAUKEE AVE,CHICAGO,IL,60618.0,02/02/2018,Canvass Re-Inspection,Pass,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...,41.932657,-87.713056,"(41.93265693660558, -87.71305610116421)"
1,2144489,EL TACONAZO JR.,EL TACONAZO JR.,2354772.0,Restaurant,Risk 1 (High),4256 W FULLERTON AVE,CHICAGO,IL,60639.0,02/02/2018,Canvass,Pass w/ Conditions,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.924508,-87.733980,"(41.92450769084945, -87.73398032565292)"
2,2144480,BRICK HOUSE BAR AND GRILL,BRICK HOUSE BAR AND GRILL,1969907.0,Restaurant,Risk 1 (High),3152-3154 W IRVING PARK RD,CHICAGO,IL,60618.0,02/02/2018,Canvass Re-Inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.954032,-87.707720,"(41.95403155441393, -87.70771986355506)"
3,2144477,THE VEGGIE GRILL,THE VEGGIE GRILL,2563666.0,NaN,Risk 1 (High),614 W DIVERSEY AVE,CHICAGO,IL,60614.0,02/02/2018,License,Not Ready,NaN,41.932917,-87.644572,"(41.93291687750861, -87.64457163885304)"
4,2144474,SMOQUE BBQ,SMOQUE BBQ,1694866.0,Restaurant,Risk 1 (High),3800 N PULASKI RD,CHICAGO,IL,60641.0,02/02/2018,Canvass,Pass w/ Conditions,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.950073,-87.727657,"(41.95007328750793, -87.72765652046023)"


In [46]:
idlookupdf = healthdf[['DBA Name','Address','Latitude','Longitude']].copy(deep=True)
idlookupdf = idlookupdf.drop_duplicates()
idlookupdf['LocId'] = range(1, len(idlookupdf) + 1)

idlookupdf.tail()

,DBA Name,Address,Latitude,Longitude,LocId
164583,"Eastside Soccer Dome, Inc.",11919 S Avenue O AVE FL,41.678517,-87.539894,28950
164612,DUNKIN DONUTS,970 W PERSHING RD,41.823504,-87.650666,28951
164621,SAFAH FOOD & LIQUOR INC,7105 S RACINE AVE,41.764896,-87.653965,28952
164636,RAINBOW GROCERY,2003 W 69TH ST,41.768328,-87.673819,28953
164644,MAKIA FOOD,2458 S CALIFORNIA AVE,41.846516,-87.695423,28954


## 0.2 Lookup Busness ID in Yelp API
We will need the exact business id to look up the business info for the project.
We need to use the lookup API to match the location to the business ID

In [83]:
api_key = 'sBOkZripW7GrsfC-YUK0in8lvw4brdRutRBA8qrUeeEdTnO_RnZok2gMWELxr-bIkaGprF65q_jT1EWSnwINomxtOQPEBLq1gsQ7yKtIRv78V9ErJP0NjRxjGvp5WnYx'
headers = {'Authorization':'Bearer '+api_key}

In [68]:
url = "https://api.yelp.com/v3/businesses/matches/best"

In [163]:
def id_lookup(name,address,lat,long,locid,df):
    '''
    Function to take request parameters and update dataframe with get info
    also saves to log each addition so if it fails I can pick up where it left off
    '''
    param = {'name': str(name),
                'address1': str(address),
                'city': 'Chicago',
                'state': 'IL',
                'country': 'US',
                'latitude': lat,
                'longitude':long}
    try:
        req = requests.get(url, params = param, headers = headers)
        req.raise_for_status()
        newdf = pd.io.json.json_normalize(req.json(),'businesses')
        newdf['LocId'] = locid
        df = df.append(newdf)
        df.to_pickle('/home/michael/chicagohealthinspections/data/idlookup.pkl')
    except:
        df = df
    return df

In [19]:
idresults = pd.read_pickle('/home/michael/chicagohealthinspections/data/idlookup.pkl')

30072

In [57]:
for index, row in idlookupdf.iterrows():
    idresults = id_lookup(row['DBA Name'],row['Address'],row['Latitude'],row['Longitude'],row['LocId'],idresults)

AttributeError: 'Series' object has no attribute 'iterrows'

In [58]:
idlink = pd.merge(idlookupdf,idresults,how='inner')
idlink = idlink[['DBA Name','Address','Latitude','Longitude','LocId','id']]
#idlink.to_pickle('/home/michael/chicagohealthinspections/data/idlink.pkl')

In [73]:
idlink.shape

(15740, 6)

In [77]:
missingids = pd.merge(idlookupdf,idlink,how='left')
missingids = missingids[pd.isnull(missingids['id'])]
missingids.shape

(13214, 6)

## 0.3 Use Id Link file to pull business info
Now that I have the business Ids, i can now look up the specific business information for matched locations

In [78]:
idlink = pd.read_pickle('/home/michael/chicagohealthinspections/data/idlink.pkl')

In [79]:
idlink.head()

,DBA Name,Address,Latitude,Longitude,LocId,id
0,TAQUERIA EL RANCHITO,2829 N MILWAUKEE AVE,41.932657,-87.713056,1,el-ranchito-restaurant-chicago-5
1,EL TACONAZO JR.,4256 W FULLERTON AVE,41.924508,-87.733980,2,el-taconazo-chicago-4
2,BRICK HOUSE BAR AND GRILL,3152-3154 W IRVING PARK RD,41.954032,-87.707720,3,brick-house-cafe-chicago
3,SMOQUE BBQ,3800 N PULASKI RD,41.950073,-87.727657,5,smoque-bbq-chicago
4,GALA BANQUETS,5639 N MILWAUKEE AVE,41.984011,-87.775172,7,gala-banquets-chicago


In [124]:
def business_lookup(businessid,df):
    '''
    Takes business Id and dataframe and appends data frame with results from business id lookup request
    '''
    try:
        url2 = 'https://api.yelp.com/v3/businesses/'
        json =  test = requests.get(url2+str(businessid), headers = headers)
        newdf = pd.io.json.json_normalize(json.json())
        df = df.append(newdf)
        df.to_pickle('/home/michael/chicagohealthinspections/data/businessinfo.pkl')
    except:
        df = df
    return df

In [131]:
businfodf = pd.read_pickle('/home/michael/chicagohealthinspections/data/businessinfo.pkl')
businfodf.shape

(14566, 28)

In [133]:
update = pd.merge(idlink,businfodf,how='left')
update = update[pd.isnull(update['name'])]
update.shape

(5, 33)

In [130]:
for index, row in update.iterrows():
    businfodf = business_lookup(row['id'],businfodf)